## 数据处理

In [5]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Alignment

In [6]:
# ---------------------客服读取数据-----------------------------
# Excel 不允许的控制字符范围  写入Excel表格时出的问题
ILLEGAL_CHARACTERS_RE = re.compile(r'[\x00-\x08\x0B-\x0C\x0E-\x1F]')
def clean_excel_df(df):
    """清理 DataFrame 中的非法 Excel 字符"""
    return df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub("", x) if isinstance(x, str) else x)
quote_pattern = re.compile(r"^(@[^\s@]+)\s+\1")
# MAPPING_FILE = "mapping.xlsx"
MAPPING_FILE = "mapping地球1.xlsx"

In [7]:
# — 0a. 读 昵称映射（real → [nick1, nick2, ...]）
df_nick = pd.read_excel(MAPPING_FILE, sheet_name="昵称映射")
# 确保都是 str，并去两端空白
df_nick["真实客服"] = df_nick["真实客服"].astype(str).str.strip()
df_nick["昵称"]    = df_nick["昵称"].astype(str).str.strip()
nickname_to_real = (
    df_nick
    .groupby("真实客服", sort=False)["昵称"]
    .apply(list)
    .to_dict()
)
# — 0b. 读 组→成员
df_grp_mem = pd.read_excel(MAPPING_FILE, sheet_name="组→成员")
df_grp_mem["客服组"] = df_grp_mem["客服组"].astype(str).str.strip()
df_grp_mem["成员"]   = df_grp_mem["成员"].astype(str).str.strip()
group_map = {
    grp: {"members": sub["成员"].tolist()}
    for grp, sub in df_grp_mem.groupby("客服组", sort=False)
}
# — 0c. 读 组→群，补充到 group_map
df_grp_cha = pd.read_excel(MAPPING_FILE, sheet_name="组→群")
# **关键**：把“群名称”都转成 str，再 strip
df_grp_cha["客服组"] = df_grp_cha["客服组"].astype(str).str.strip()
df_grp_cha["群名称"] = df_grp_cha["群名称"].astype(str).str.strip()
for grp, sub in df_grp_cha.groupby("客服组", sort=False):
    groups = sub["群名称"].tolist()
    if grp in group_map:
        group_map[grp]["groups"] = groups
    else:
        # 如果有新的组，就给个空 members，免得 KeyError
        group_map[grp] = {"members": [], "groups": groups}
# ———— 1. 构造客服 → 负责群 列表 ————
rep_to_groups = {}
for info in group_map.values():
    # 这里用 info['groups']，此时都是字符串
    for rep in info['members']:
        rep_to_groups.setdefault(rep, []).extend(info['groups'])
# 去重、排序
for rep, gl in rep_to_groups.items():
    rep_to_groups[rep] = sorted(set(gl))

In [8]:
import re
import pandas as pd
# ———————————————— 1. 解析 + 标注 DataFrame ————————————————
def load_and_process(filepath: str, nickname_to_real: dict) -> pd.DataFrame:
    # 读取 txt → list[str]
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        raw = f.read().splitlines()
    recs, cur_grp, cur_obj = [], None, None
    pat = re.compile(r"(\d{4}-\d{2}-\d{2} \d{1,2}:\d{2}:\d{2})\s+(.+)")
    i = 0
    while i < len(raw):
        line = raw[i].strip()
        if line.startswith("消息分组:"):
            cur_grp = line.split(":", 1)[1].strip(); i += 1; continue
        if line.startswith("消息对象:"):
            cur_obj = line.split(":", 1)[1].strip(); i += 1; continue
        m = pat.match(line)
        if m and cur_obj:
            t, sender = m.groups(); i += 1
            content = raw[i].strip() if i < len(raw) else ""
            recs.append({
                "消息分组": cur_grp,
                "聊天对象/群": cur_obj,
                "时间": t,
                "发言人": sender,
                "消息内容": content
            })
        i += 1
    df = pd.DataFrame(recs)
    df.drop_duplicates(inplace=True)
    df['时间'] = pd.to_datetime(df['时间'], errors='coerce')
    # 标注 使用人 & 真实客服
    df['使用人'] = df['发言人']
    df['真实客服'] = None
    for real, nicks in nickname_to_real.items():
        pat = "|".join(re.escape(x) for x in nicks + [real])
        df.loc[df['使用人'].str.contains(pat, na=False), '真实客服'] = real 
    # 提取 speaker_id
    df['speaker_id'] = (
        df['发言人']
          .str.extract(r'\((\d+)\)')
          .fillna(df['发言人'])
    )
    # 去掉撤回消息 & 指定群
    df = df[~df['speaker_id'].isin(['10000', '1000000','系统消息(1000000)'])]
    df = df[~df['聊天对象/群'].isin(['青瓷客服打卡群'])]
    return df


In [9]:
pathtxt="《欢迎来到地球》测试1群.txt"
df01 = load_and_process(pathtxt, nickname_to_real)
print(len(df01))

95310


In [10]:
import pandas as pd

# 你的原始DataFrame，假设叫df
# 创建映射关系
speaker_map = {
    "16186514": "研发peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师"
}

# 添加一列“研发”，根据speaker_id映射值填充
df01["研发"] = df01["speaker_id"].map(speaker_map)
print(len(df01))
df01[:5]

95310


,消息分组,聊天对象/群,时间,发言人,消息内容,使用人,真实客服,speaker_id,研发
5,我管理的群聊,《欢迎来到地球》测试1群,2025-05-27 09:59:44,薏米(1090838043),小迪与除秋盼外的其他人均不是好友关系，请注意隐私安全。,薏米(1090838043),薏米,1090838043,NaN
7,我管理的群聊,《欢迎来到地球》测试1群,2025-06-30 16:18:20,薏米(1090838043),[图片],薏米(1090838043),薏米,1090838043,NaN
10,我管理的群聊,《欢迎来到地球》测试1群,2025-07-01 10:04:19,大薏米地字⑤号(3533354001),,大薏米地字⑤号(3533354001),薏米,3533354001,NaN
51,我管理的群聊,《欢迎来到地球》测试1群,2025-07-01 14:25:42,薏米(1090838043),[图片],薏米(1090838043),薏米,1090838043,NaN
52,我管理的群聊,《欢迎来到地球》测试1群,2025-07-01 14:25:49,薏米(1090838043),好多人哇~,薏米(1090838043),薏米,1090838043,NaN


In [11]:
####数据清洗

# 1. 处理“空行”或“[表情]”的情况
mask_empty_or_emoji = df01["消息内容"].astype(str).str.strip().isin(["", "[表情]"])
# 2. 处理灌水词（可自行扩展）
spam_words = ["+1", "冲", "蹲", "up", "哈", "嘿", "哦"]
mask_spam = df01["消息内容"].isin(spam_words)
# 合并两个条件
mask_to_drop = mask_empty_or_emoji | mask_spam
# 删除这些无效行
df_cleaned = df01[~mask_to_drop].copy()
df01=df_cleaned
print(len(df01))

93054


In [12]:
df01["时间"] = pd.to_datetime(df01["时间"], errors="coerce")

# 设置时间范围（覆盖两天）
start_time = pd.to_datetime("2025-08-06 00:00:00")
end_time   = pd.to_datetime("2025-08-07 00:00:00")

# 筛选两天的数据
filtered_df = df01[(df01["时间"] >= start_time) & (df01["时间"] < end_time)]
print(len(filtered_df))
df=filtered_df


5825


In [13]:
df_selected = df[["时间", "发言人", "消息内容", "真实客服", "研发","speaker_id"]]
df_selected[:5]

,时间,发言人,消息内容,真实客服,研发,speaker_id
49044,2025-08-06 00:00:07,大头娃娃(2441066104),挺一般的 看电视居然要VIP,None,NaN,2441066104
49045,2025-08-06 00:00:08,大头娃娃(2441066104),[图片],None,NaN,2441066104
49046,2025-08-06 00:00:25,大头娃娃(2441066104),我昨天住的都自带VIP和智能家居的,None,NaN,2441066104
49047,2025-08-06 00:57:23,香草大炮(1197844058),现在都是了,None,NaN,1197844058
49048,2025-08-06 00:57:54,香草大炮(1197844058),我大学旁边的打炮旅馆充电线都要钱,None,NaN,1197844058


In [14]:
df_selected = df[["时间", "发言人", "消息内容", "真实客服", "研发","speaker_id"]]
def identify_speaker(row):
    if pd.notna(row["研发"]):
        return "研发ID"
    elif pd.notna(row["真实客服"]):
        return "客服ID"
    else:
        return "玩家ID"

# 应用函数添加新列
df_selected["玩家/客服/研发"] = df_selected.apply(identify_speaker, axis=1)
df_selected[:5]

C:\Users\wujn\AppData\Local\Temp\ipykernel_32524\3738544442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["玩家/客服/研发"] = df_selected.apply(identify_speaker, axis=1)


,时间,发言人,消息内容,真实客服,研发,speaker_id,玩家/客服/研发
49044,2025-08-06 00:00:07,大头娃娃(2441066104),挺一般的 看电视居然要VIP,None,NaN,2441066104,玩家ID
49045,2025-08-06 00:00:08,大头娃娃(2441066104),[图片],None,NaN,2441066104,玩家ID
49046,2025-08-06 00:00:25,大头娃娃(2441066104),我昨天住的都自带VIP和智能家居的,None,NaN,2441066104,玩家ID
49047,2025-08-06 00:57:23,香草大炮(1197844058),现在都是了,None,NaN,1197844058,玩家ID
49048,2025-08-06 00:57:54,香草大炮(1197844058),我大学旁边的打炮旅馆充电线都要钱,None,NaN,1197844058,玩家ID


In [15]:
# 确保时间列是 datetime 类型
df_selected["时间"] = pd.to_datetime(df_selected["时间"])

# 新增“日期”和“时分秒”两列
df_selected["日期"] = df_selected["时间"].dt.date
df_selected["时分秒"] = df_selected["时间"].dt.time
df_selected01=df_selected[["日期","时分秒", "玩家/客服/研发", "发言人", "消息内容","speaker_id"]]
df_selected01[:5]

C:\Users\wujn\AppData\Local\Temp\ipykernel_32524\1543869772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["时间"] = pd.to_datetime(df_selected["时间"])
C:\Users\wujn\AppData\Local\Temp\ipykernel_32524\1543869772.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["日期"] = df_selected["时间"].dt.date


,日期,时分秒,玩家/客服/研发,发言人,消息内容,speaker_id
49044,2025-08-06,00:00:07,玩家ID,大头娃娃(2441066104),挺一般的 看电视居然要VIP,2441066104
49045,2025-08-06,00:00:08,玩家ID,大头娃娃(2441066104),[图片],2441066104
49046,2025-08-06,00:00:25,玩家ID,大头娃娃(2441066104),我昨天住的都自带VIP和智能家居的,2441066104
49047,2025-08-06,00:57:23,玩家ID,香草大炮(1197844058),现在都是了,1197844058
49048,2025-08-06,00:57:54,玩家ID,香草大炮(1197844058),我大学旁边的打炮旅馆充电线都要钱,1197844058


In [16]:
import json

jsonl_lines01 = []  # 这里存放 JSONL 格式的字符串

for _, row in df_selected01.iterrows():
    row_type = row["玩家/客服/研发"]
    发言日期 = str(row["日期"])
    发言时间 = str(row["时分秒"])
    speaker_id = str(row["发言人"])
    content = str(row["消息内容"]).strip()

    if row_type == "玩家ID":
        line = {
            "发言日期": 发言日期,
            "发言时间": 发言时间,
            "玩家ID": speaker_id,
            "玩家消息": content
        }
    elif row_type == "客服ID":
        line = {
            "发言日期": 发言日期,
            "发言时间": 发言时间,
            "客服ID": speaker_id,
            "客服消息": content
        }
    elif row_type == "研发ID":
        line = {
            "发言日期": 发言日期,
            "发言时间": 发言时间,
            "研发ID": speaker_id,
            "研发消息": content
        }
    else:
        continue  # 跳过无法识别的

    # 转换为 JSON 字符串并存入列表，每行一个 JSON
    jsonl_lines01.append(json.dumps(line, ensure_ascii=False))


In [18]:
len(jsonl_lines01)

5825

## 大模型分类

## 定义

In [14]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：对保留内容进行 5 类意图分类（输出带“意图分类”的 JSON 行）
- 结果按 sheet（体感反馈/疑惑不解和问题询问/玩家建议和灵感/情绪输出/问题反馈）写入 Excel
"""

import os, re, json, time
import typing as T
import requests
from pathlib import Path
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill

In [15]:
def load_system_prompt(path: Path) -> str:
    if not path.exists():
        raise FileNotFoundError(f"Prompt file not found: {path}")
    return path.read_text(encoding="utf-8")

def build_user_prompt_filter(json_lines: T.List[str]) -> str:
    # 模型#1：筛掉非游戏相关，只输出相关 JSON 行（原样）
    return (
        "以下是若干玩家/客服/研发的发言记录，请根据系统提示中规则，"
        "判断哪些是【与游戏内容相关】的发言，保留这些 JSON 行，不相关的忽略。"
        "请仅输出【相关发言的原始 JSON 行】，严格保持格式不变。\n\n"
        "【输入】：\n" + "\n".join(json_lines)
    )

def build_user_prompt_classify(jsonl_block: str) -> str:
    # 模型#2：对已筛选的相关 JSON 行（原样）进行分类，追加“意图分类”键
    return (
       "以下是输入数据（JSONL 格式，每行一个发言对象）：\n\n"
        "请仅输出【 JSON 行】，\n\n"
        "【输入】：\n" + jsonl_block
    )

def call_ark_chat_completions(
    api_url: str,
    api_key: str,
    model: str,
    system_prompt: str,
    user_prompt: str,
    temperature: float = 0.3,
    max_tokens: int = 32700,
    timeout: int = 600,
    retries: int = 2,
) -> str:
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    last_err = None
    for attempt in range(retries + 1):
        try:
            resp = requests.post(api_url, headers=headers, json=payload, timeout=timeout)
            if resp.status_code != 200:
                raise RuntimeError(f"HTTP {resp.status_code}: {resp.text}")
            data = resp.json()
            return data["choices"][0]["message"]["content"]
        except Exception as e:
            last_err = e
            time.sleep(1.2 * (attempt + 1))
    raise RuntimeError(f"Ark API 调用失败: {last_err}")

def extract_valid_json_lines(text: str) -> T.List[str]:
    """
    把模型输出里的纯 JSON 行提取出来（鲁棒处理）：
    - 逐行判断：以 { 开头 且 以 } 结尾，则认为是一个 JSON 对象行
    - 也能容忍前后多余空行或解释文字（会被忽略）
    """
    lines = []
    for line in text.splitlines():
        s = line.strip()
        if not s:
            continue
        if s.startswith("{") and s.endswith("}"):
            lines.append(s)
    return lines



def jsonl_to_dataframe_with_intent(jsonl_text: str) -> pd.DataFrame:
    """
    模型#2 的输出应为 JSONL（每行一个 JSON，含 发言日期/发言时间/玩家ID/玩家消息/意图分类）。
    转成 DataFrame，并合并“时间”列。
    """
    pure_lines = extract_valid_json_lines(jsonl_text)
    if not pure_lines:
        return pd.DataFrame(columns=["发言时间", "玩家ID", "玩家消息", "分类标签"])
    rows = [json.loads(line) for line in pure_lines]
    df = pd.DataFrame(rows)

    # 只保留这三列 + 意图分类
    keep_cols = ["发言时间", "玩家ID", "玩家消息", "分类标签"]
    for c in keep_cols:
        if c not in df.columns:
            df[c] = None
    df = df[keep_cols].copy()
    return df




In [16]:

SHEET_NAMES = ["体验反馈", "疑惑询问", "建议灵感", "情绪输出", "问题反馈"]
HEADERS = ["发言时间", "玩家ID", "玩家消息"]

# 样式配置
FONT_NAME = "微雅软黑"         # 若本机没有，可改 "微软雅黑"
HEADER_FONT_SIZE = 12
BODY_FONT_SIZE   = 11
HEADER_FILL      = "DDDDDD"     # 灰底
HEADER_ROW_HEIGHT = 24
COL_WIDTHS = [21, 30, 95]       # 时间 / 玩家ID / 玩家消息

def create_intent_excel_styled(filename: str):
    wb = Workbook()
    ws0 = wb.active
    ws0.title = SHEET_NAMES[0]

    # 先把 5 个 sheet 与表头建好
    for idx, name in enumerate(SHEET_NAMES):
        ws = wb[name] if idx == 0 else wb.create_sheet(title=name)
        # 表头
        ws.append(HEADERS)
        # 列宽
        for col_idx, w in enumerate(COL_WIDTHS, start=1):
            ws.column_dimensions[get_column_letter(col_idx)].width = w
        # 冻结首行
        ws.freeze_panes = "A2"
        # 表头样式
        for c in range(1, len(HEADERS)+1):
            cell = ws.cell(row=1, column=c)
            cell.font = Font(name=FONT_NAME, size=HEADER_FONT_SIZE, bold=True)
            cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
            cell.fill = PatternFill("solid", fgColor=HEADER_FILL)
        # 表头行高
        ws.row_dimensions[1].height = HEADER_ROW_HEIGHT

    wb.save(filename)
    print(f"✅ 创建并套样式：{filename}")


In [17]:


CA_TO_SHEET = {1:"体验反馈", 2:"疑惑询问", 3:"建议灵感", 4:"情绪输出", 5:"问题反馈"}

# 正文样式（每次 append 时给新行套上）
_BODY_FONT = Font(name=FONT_NAME, size=BODY_FONT_SIZE)
_BODY_ALIGN = Alignment(vertical="center", wrap_text=True)
_THIN_BORDER = Border(
    left=Side(style="thin", color="000000"),
    right=Side(style="thin", color="000000"),
    top=Side(style="thin", color="000000"),
    bottom=Side(style="thin", color="000000"),
)

def append_df_to_excel_by_ca_styled(df: pd.DataFrame, excel_path: str):
    """
    df 需包含：['发言时间','玩家ID','玩家消息','分类标签']，其中 CA ∈ {1..5}
    作用：按 CA 分发到对应 sheet，并给新追加行套正文样式（自动换行、细边框、字体）。
    """
    if df.empty:
        return

    wb = load_workbook(excel_path)

    # 只取三列 + CA
    need = ["发言时间", "玩家ID", "玩家消息", "分类标签"]
    for c in need:
        if c not in df.columns:
            df[c] = pd.NA
    df3 = df[need].copy()

    for ca, g in df3.groupby("分类标签"):
        try:
            ca = int(ca)
        except:
            continue
        sheet = CA_TO_SHEET.get(ca)
        if not sheet or sheet not in wb.sheetnames:
            continue
        ws = wb[sheet]

        for _, r in g.iterrows():
            # 追加一行
            ws.append([r["发言时间"], r["玩家ID"], r["玩家消息"]])
            # 给刚追加的最后一行套正文样式 + 边框
            last_row = ws.max_row
            for col in range(1, len(HEADERS)+1):
                cell = ws.cell(row=last_row, column=col)
                cell.font = _BODY_FONT
                cell.alignment = _BODY_ALIGN
                cell.border = _THIN_BORDER

    wb.save(excel_path)


## 设置参数

In [18]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
# API_KEY = "fcb27d69-1f94-46c7-b088-49f909e2f1d1"
# MODEL_ID  = "ep-20250630184125-mlgg8"     # 你的模型ID
API_KEY = "51fb7806-f343-4767-8152-40a15f16838c" #辰少密钥
MODEL_ID = "ep-20250604162650-qfn2d"  #辰少 ds

PROMPT_MD_PATH01 = Path("提示词1.md")      # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("提示词2.md")      # 模型#2 system 提示词（做分类）

EXCEL_FILE       = Path("意图分类结果_总表test7.xlsx")   # 输出 Excel
BATCH_SIZE       = 100
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.25
MAX_TOKENS       = 32700
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

In [20]:
from tqdm import tqdm  # ✅ 新增库

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做分类
create_intent_excel_styled(EXCEL_FILE)


total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
    # return

total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

# ✅ 用 tqdm 包装循环
for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
            # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
                api_url=API_URL,
                api_key=API_KEY,
                model=MODEL_ID,
                system_prompt=system_prompt01,
                user_prompt=user_prompt1,
                temperature=TEMPERATURE,
                max_tokens=MAX_TOKENS,
                timeout=TIMEOUT_SEC,
                retries=RETRIES,
            )
        print("第一步完成")
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

            # --- 模型 #2：分类 ---
        user_prompt2 = build_user_prompt_classify(output_filter)
        output_classify = call_ark_chat_completions(
                api_url=API_URL,
                api_key=API_KEY,
                model=MODEL_ID,
                system_prompt=system_prompt02,
                user_prompt=user_prompt2,
                temperature=TEMPERATURE,
                max_tokens=MAX_TOKENS,
                timeout=TIMEOUT_SEC,
                retries=RETRIES,
            )

        # 转 DataFrame + 写 Excel
        df_batch = jsonl_to_dataframe_with_intent(output_classify)
        append_df_to_excel_by_ca_styled(df_batch, EXCEL_FILE)  # ← 我上一条给你的写入函数

        tqdm.write(f"[批次 {b+1}] ✅ 写入 {len(df_batch)} 条。")

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")


✅ 创建并套样式：意图分类结果_总表test7.xlsx
准备处理 5825 条，共 59 批（每批 100 条）。


🔥 批处理进度:   0%|                                                                            | 0/59 [00:00<?, ?批/s]

第一步完成


🔥 批处理进度:   0%|                                                                            | 0/59 [09:05<?, ?批/s]      

[批次 1] ✅ 写入 49 条。


...

第一步完成


🔥 批处理进度:  98%|█████████████████████████████████████████████████████████████▉ | 58/59 [10:22:40<11:10, 670.78s/批]      

[批次 59] ✅ 写入 25 条。


🔥 批处理进度: 100%|███████████████████████████████████████████████████████████████| 59/59 [10:22:41<00:00, 633.25s/批]


✅ 全部批次处理完成！
